In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized_full")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="indy7"
DATASET_TRAIN = ['20210209-193147', '20210209-213935', '20210209-224826', '20210210-000355']
DATASET_TEST = ['20210210-011407']
ARM_FOLDER = ARM10_FOLDER
FULL_SHAPE = (60,60,60)

ROBOT_DATA_ROOT = os.path.join(DATA_PATH, ROBOT_TYPE_NAME)
dataset_train = []
for dataset in DATASET_TRAIN:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER)))
    for file in file_list:
        dataset_train.append(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER, file))
        
dataset_test = []
for dataset in DATASET_TEST:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER)))
    for file in file_list:
        dataset_test.append(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER, file))

In [4]:
def load_data(data_path):
    full_data = load_pickle(data_path)
    full_tar_idx = full_data[b'tar']
    full_tool_idx = full_data[b'tool']
    reach_lb = full_data[b'reach']
    retrieve_lb = full_data[b'retrieve']
    full_tool_img = np.zeros(FULL_SHAPE)
    full_tool_img[np.unravel_index(full_tool_idx, shape=FULL_SHAPE)] = 1
    full_tar_img = np.zeros(FULL_SHAPE)
    full_tar_img[np.unravel_index(full_tar_idx, shape=FULL_SHAPE)] = 1
    full_img = np.stack([full_tool_img, full_tar_img], axis=-1)
    label = np.array([reach_lb, retrieve_lb])
    return full_img, label

## Train

In [5]:
from pkg.planning.filtering.lattice_model_full import *

# Create an instance of the model
model = ResNetModelTP_FULL()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [6]:
current_time = get_now()
logpath = os.path.join(LAT_MODEL_PATH, current_time)
try_mkdir(logpath)
train_log_dir = os.path.join(logpath, 'train')
test_log_dir = os.path.join(logpath, 'test')
model_log_dir = os.path.join(logpath, 'model_{}/')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
shutil.copy(os.path.join(PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model.py' ), logpath)
print(f'Log path: {logpath}')

Log path: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617


In [7]:
EPOCHS_S = 0
EPOCHS_E = 5
BATCH_SIZE = 16
LOG_STEP = 100
N_train = len(dataset_train)
N_test = len(dataset_test)

for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(dataset_train)
    i_step = 0
    data_batch, label_batch = [], []
    for data_path in dataset_train:
        i_step += 1
        full_img, label = load_data(data_path)
        data_batch.append(full_img)
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            data_batch = np.array(data_batch)
            label_batch = np.array(label_batch, dtype=np.int)
            train_step(data_batch, label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    i_step = 0
    data_batch, label_batch = [], []
    for data_path in dataset_test:
        i_step += 1
        full_img, label = load_data(data_path)
        data_batch.append(full_img)
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            data_batch = np.array(data_batch)
            label_batch = np.array(label_batch, dtype=np.int)
            test_step(data_batch, label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            
    model.save(model_log_dir.format(epoch + 1))

    print("")
    print("=================================================================")
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
    print("=================================================================")
    print("")

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_1/assets



Epoch 1, Loss: 0.7211742401123047, Accuracy: 52.17499542236328, Test Loss: 0.6880883574485779, Test Accuracy: 55.79637145996094



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_2/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_4/assets



Epoch 4, Loss: 0.7007961273193359, Accuracy: 53.63749694824219, Test Loss: 0.6884286999702454, Test Accuracy: 55.64516067504883



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_5/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized_full/20210210-031617/model_5/assets



Epoch 5, Loss: 0.6982054710388184, Accuracy: 53.57499694824219, Test Loss: 0.8109462857246399, Test Accuracy: 52.21773910522461



## Load & test

In [5]:
last_model = sorted(os.listdir(LAT_MODEL_PATH))[-1]

In [6]:
logpath = os.path.join(LAT_MODEL_PATH, last_model)
last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
model_log_dir = os.path.join(logpath, last_save)

In [7]:
import tensorflow as tf
model = tf.keras.models.load_model(model_log_dir)

In [8]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
loss_object = tf.keras.losses.BinaryCrossentropy()
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

@tf.function
def inference(images):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    return predictions

In [9]:
BATCH_SIZE = 16
LOG_STEP = 100
N_test = len(dataset_test)
# Reset the metrics at the start of the next epoch
test_loss.reset_states()
test_accuracy.reset_states()

i_step = 0
data_batch, label_batch = [], []
for data_path in dataset_test:
    i_step += 1
    full_img, label = load_data(data_path)
    data_batch.append(full_img)
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        data_batch = np.array(data_batch)
        label_batch = np.array(label_batch, dtype=np.int)
        test_step(data_batch, label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

print("")
print("=================================================================")
print(
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
)
print("=================================================================")
print("")

test step - 1000/1004        
Test Loss: 0.8109458088874817, Test Accuracy: 52.21773910522461



In [10]:
# data_path = dataset_train[0]
# full_data = load_pickle(data_path)
# full_tar_idx = full_data[b'tar']
# full_tool_idx = full_data[b'tool']
# save_json("tool.json", np.array(np.unravel_index(full_tool_idx, shape=FULL_SHAPE)).transpose())
# save_json("tar.json", np.array(np.unravel_index(full_tar_idx, shape=FULL_SHAPE)).transpose())